# Language Models


In the following we make some basic implementations of n-gram language models, a few simple smoothing methods and train and evaluate our models on a small corpus and finally build a small application.

Let us start with a class to store n-gram counts. We have to store the counts for many word sequences (n grams) and should be able to retrieve them efficiently. Here we use a prefix tree for this purpose.

In [1]:
from  collections import Counter

class node:
    count = 0
    branches = {}
    
    def __init__(self):
        self.count = 0
        self.branches = {}
    
class ngramcounts:
    
    tree = node()
    maxlen = 0
    
    def __init__(self):
        self.tree = node()
    
    def add(self,words,amount=1):
        if len(words) > self.maxlen:
            self.maxlen = len(words)
        current = self.tree
        current.count += amount
        for w in words:
            nxt = current.branches.get(w,node())
            nxt.count += amount
            current.branches[w] = nxt
            current = nxt
            
    def getcount(self,words):
        current = self.tree
        for w in words:
            current = current.branches.get(w.lower(),node())
            if current.count == 0:
                break
        return current.count
    
    def getsuccessors(self,words):
        current = self.tree
        for w in words:
            current = current.branches.get(w,node())
            if current.count == 0:
                break
        successors = current.branches
        return Counter({w:successors[w].count for w in successors})
    
    def allngrams(self,maxdepth):
        stack = []
        ng =[]
        for word in self.tree.branches:
            ndword = self.tree.branches[word]
            stack.append(([word],ndword.count,ndword.branches))
        
        while len(stack) > 0:
            ng,cnt,tree = stack[-1]
            yield(ng,cnt)
            stack = stack[:-1]
            if len(ng) < maxdepth:
                for word in tree:
                    n = tree[word]
                    stack.append((ng+[word],n.count,n.branches))

#### Testing the class

Let us just write some code to test the class

In [2]:
test = ngramcounts()
test.add(['a','b','c'],2)
test.add(['a','b','d'])
test.add(['b','d','e'])

In [3]:
for ng in test.allngrams(3):
    print(ng)

(['b'], 1)
(['b', 'd'], 1)
(['b', 'd', 'e'], 1)
(['a'], 3)
(['a', 'b'], 3)
(['a', 'b', 'd'], 1)
(['a', 'b', 'c'], 2)


In [4]:
for ng in test.allngrams(2):
    print(ng)

(['b'], 1)
(['b', 'd'], 1)
(['a'], 3)
(['a', 'b'], 3)


In [5]:
test.getcount(['a','b'])

3

In [6]:
test.getsuccessors(['a','b'])

Counter({'c': 2, 'd': 1})

In [7]:
test.getsuccessors([])

Counter({'a': 3, 'b': 1})

## Read n-Grams from a Corpus

We see a corpus as one long list of words. We use a sliding window to collect n-grams. In this way we might loss some shorter sequences at the beginning and the end. For a corpus of 1 Mio words this is, however, neglectible.

In [8]:
from collections import deque

def collect_ngram(words,n):
    counts = ngramcounts()
    last_n = deque([],n)
    for w in words:
        w = w.lower()
        last_n.append(w)
        if len(last_n) == n:
            counts.add(list(last_n))
    return counts
            

We use the Brown corpus that is available in the NLTK. Eventually you have to download this first. We excluse the firts 10,000 words. So we can use this part later for testing. Since the corpus is sorted over text genres, this is not the split that will give us the best results, but still better results than a test on a completely different corpus!

In [9]:
import nltk

nltk.download('brown')

[nltk_data] Downloading package brown to
[nltk_data]     /Users/sruthysanthosh/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [10]:
from nltk.corpus import brown

len(brown.words())

1161192

In [11]:
ngram_cnt = collect_ngram(brown.words()[10000:],4)

Let us have a look at the colleted counts in order to see wheter we have plausible data and nothing went wrong.

In [12]:
ngram_cnt.getcount(['the','old','testament'])

5

In [13]:
ngram_cnt.getsuccessors(['the','old']).most_common()

[('man', 36),
 ('woman', 10),
 ('days', 7),
 ('testament', 5),
 ('chinese', 5),
 ('south', 4),
 ('church', 4),
 ('west', 3),
 ('school', 3),
 ('southern', 3),
 ('order', 3),
 ('lady', 3),
 ('ones', 3),
 ("man's", 3),
 ('phrase', 2),
 ('grad', 2),
 ('story', 2),
 ('house', 2),
 ('way', 2),
 ('roman', 2),
 (',', 2),
 ('monk', 2),
 ('country', 2),
 ('town', 1),
 ('``', 1),
 ('vic', 1),
 ('picker', 1),
 ('chisholm', 1),
 ('silent', 1),
 ('adam', 1),
 ('suffrage', 1),
 ('resolve', 1),
 ('couples', 1),
 ('injury', 1),
 ('new', 1),
 ('type', 1),
 ('scenes', 1),
 ('paths', 1),
 ('campus', 1),
 ('player', 1),
 ('under', 1),
 ('boy', 1),
 ('production', 1),
 ('war', 1),
 ('eighteenth-century', 1),
 ('plays', 1),
 ('tradition', 1),
 ('consummate', 1),
 ('mind', 1),
 ('puritan', 1),
 ('interest', 1),
 ('remington', 1),
 ('model', 1),
 ('wells', 1),
 ('whaling', 1),
 ('frontier', 1),
 ('section', 1),
 ('covered', 1),
 ('bridge', 1),
 ('beverly', 1),
 ('john', 1),
 ('stand', 1),
 ('established', 1),

In [14]:
ngram_cnt.getsuccessors([]).most_common()

[('the', 69296),
 (',', 57930),
 ('.', 48969),
 ('of', 36097),
 ('and', 28679),
 ('to', 25893),
 ('a', 22988),
 ('in', 21142),
 ('that', 10495),
 ('is', 10054),
 ('was', 9754),
 ('he', 9477),
 ('for', 9379),
 ('``', 8734),
 ('it', 8707),
 ("''", 8690),
 ('with', 7254),
 ('as', 7200),
 ('his', 6961),
 ('on', 6673),
 ('be', 6308),
 (';', 5564),
 ('at', 5342),
 ('by', 5235),
 ('i', 5157),
 ('had', 5113),
 ('this', 5108),
 ('?', 4689),
 ('not', 4575),
 ('are', 4375),
 ('but', 4359),
 ('from', 4345),
 ('or', 4184),
 ('have', 3918),
 ('an', 3706),
 ('they', 3604),
 ('which', 3531),
 ('--', 3410),
 ('you', 3282),
 ('one', 3268),
 ('were', 3267),
 ('her', 3036),
 ('all', 2993),
 ('she', 2860),
 ('there', 2701),
 ('their', 2658),
 ('we', 2649),
 ('would', 2634),
 ('him', 2618),
 (')', 2451),
 ('been', 2438),
 ('(', 2420),
 ('has', 2389),
 ('when', 2321),
 ('who', 2227),
 ('will', 2209),
 ('more', 2190),
 ('if', 2187),
 ('no', 2123),
 ('out', 2084),
 ('so', 1980),
 ('what', 1902),
 ('said', 1885

## Generate Random Text

Let us generate some random text. We generate each time a random word, but shoose only from words that have been found in the corpus as a follower of the previous n-1 words. The probability to choose a word should correspond to the distribution in the corpus.

In [15]:
import random
import itertools
import re

def generate_rand(counts,start,n,maxln):
    if n > counts.maxlen:
        n = counts.maxlen
    words = start
    nrOfWords = len(start)
    if n > 1:
        last = words[-(n-1):]
    else:
        last = []
    while nrOfWords < 2*maxln:
        candidates = counts.getsuccessors(last)
        #stop if no successors are found or a nice ending was reached
        if len(candidates) == 0 or (nrOfWords > 0.9*maxln and words[-1] in ".!?"):
            break
        i = random.randrange(sum(candidates.values()))
        w = next(itertools.islice(candidates.elements(), i, None))
        words.append(w)
        nrOfWords += 1
        if n > 1:
            last = words[-(n-1):]
        else:
            last = []
     
    #Make the text look nice
    text = words[0].capitalize()
    for i in range(1,len(words)):
        word = words[i]
        if word[0] not in ".,;:!?)" and words[i-1] != '(':
            text += " "
        if words[i-1] == '.' or words[i-1] == '!' or words[i-1] == '?':
            text += word.capitalize()
        else:
            text += word
            
    return text

In [16]:
generate_rand(ngram_cnt,[],1,20)


"Morning `` access sansom skirt without too for the on, long-range u.s. scared switching,. Little feel it '' of a pieces her accordingly for, anyone secondary 15 ``, the had to fall wording danced important"

In [17]:
generate_rand(ngram_cnt,[],2,30)

"Economic and the piano lessons, and the most remarkable conviction that congress, robinson cut his hands worked perfectly free and a far better than interest of an infinite wisdom and as `` salami ''."

In [18]:
generate_rand(ngram_cnt,[],3,30)

". The other side '' cropped up again in a bomber, each of her widowed mother, because excess body fat hampers his breathing. In what he hates, that is, i don't care how they're handled, mickey mantle and willie mays, the bbb has reported knowlton as saying: `` thanks '', rachel"

In [19]:
generate_rand(ngram_cnt,[],4,30)

"Flew beautifully '', said mr. skyros earnestly, `` just forget it ''. `` what were they doing here ''?? The selection of fellows from the latin american front, the first five minutes of this interview it is apparent to the therapist that `` everybody '', indicating to the linguist that this is,"

In [20]:
generate_rand(ngram_cnt,['you','may'],4,30)


"You may deduct reasonable wages you paid to him for scouting duty, most of us when we want to know what the dummies will do '' -- he interrupted himself, looking around."

## Language Models

Let us come to the language models: try to predict the next word. Here we have mainly to deal with sparsity and smoothing the model.

First we read the test data. Note, that most models can deal with out of vocabulary words, but have to know how many unknown words we have! Thus we need to collect our entire vocabulary over test and training data as well.

In [21]:
#from nltk.corpus import webtext
#testtext = webtext.words()[:100000]
testtext = brown.words()[:10000]

vocab = set(testtext+brown.words())
vocab = set([w.lower() for w in vocab])

In [22]:
len(vocab)

49815

We start with a basic class without smoothing.

If we build an n-gram model we will base the prediction, of course, on n-grams. If less preceding text is givern smalle sequences will be used.

In [23]:
class langmodel:
    
    def __init__(self,vocab,counts,n):
        self.n = n
        self.rawcounts = counts
            
        
    def prob(self,w,prev):
        if self.n == 1:
            prev = []
        else:
            prev = prev[-self.n+1:]
        n_seq = self.rawcounts.getcount(prev+[w])
        if n_seq == 0:
            return 0
        else:
            return n_seq/self.rawcounts.getcount(prev)


In [24]:
unsmoothed2 = langmodel(vocab,ngram_cnt,2)

print(unsmoothed2.prob('testament',['the','old']))
print(unsmoothed2.prob('man',['the','old']))
print(unsmoothed2.prob('tower',['the','old']))
print(unsmoothed2.prob('twer',['the','ldpuio']))

0.010606060606060607
0.10151515151515152
0
0


In [25]:
unsmoothed3 = langmodel(vocab,ngram_cnt,3)

print(unsmoothed3.prob('testament',['the','old']))
print(unsmoothed3.prob('man',['the','old']))
print(unsmoothed3.prob('tower',['the','old']))
print(unsmoothed3.prob('twer',['the','ldpuio']))

0.02145922746781116
0.15450643776824036
0
0


In [26]:
class equal(langmodel):
    
    def __init__(self,vocab,counts):
            self.V = len(vocab) 
        
    def prob(self,w,prev):
        return 1/self.V

In [27]:
eq_model = equal(vocab,ngram_cnt)

print(eq_model.prob('testament',['the','old']))
print(eq_model.prob('man',['the','old']))
print(eq_model.prob('tower',['the','old']))
print(eq_model.prob('twer',['the','ldpuio']))

2.007427481682224e-05
2.007427481682224e-05
2.007427481682224e-05
2.007427481682224e-05


### Laplace Smoothing

In [28]:
class laplace(langmodel):
    
    def __init__(self,vocab,counts,n):
            self.n = n
            self.rawcounts = counts
            self.V = len(vocab) 
        
    def prob(self,w,prev):
        if self.n == 1:
            prev = []
        else:
            prev = prev[-self.n+1:]
        n_seq = self.rawcounts.getcount(prev+[w])
        n_prev = self.rawcounts.getcount(prev)
        return (n_seq + 1)/(n_prev+self.V)

In [29]:
laplace_model2 = laplace(vocab,ngram_cnt,2)

print(laplace_model2.prob('testament',['the','old']))
print(laplace_model2.prob('man',['the','old']))
print(laplace_model2.prob('tower',['the','old']))
print(laplace_model2.prob('twer',['the','ldpuio']))

0.00015849430411094602
0.0013472015849430412
1.9811788013868253e-05
2.007427481682224e-05


In [30]:
laplace_model3 = laplace(vocab,ngram_cnt,3)

print(laplace_model3.prob('testament',['the','old']))
print(laplace_model3.prob('man',['the','old']))
print(laplace_model3.prob('tower',['the','old']))
print(laplace_model3.prob('twer',['the','ldpuio']))

0.00011988491048593351
0.0007392902813299233
1.9980818414322252e-05
2.007427481682224e-05


### Lidstone Smoothing

In [31]:
class lidstone(langmodel):
    
    def __init__(self,vocab,counts,n,labda):
            self.n = n
            self.rawcounts = counts
            self.lV = labda * len(vocab) 
            self.labda = labda
        
    def prob(self,w,prev):
        if self.n == 1:
            prev = []
        else:
            prev = prev[-self.n+1:]
        n_seq = self.rawcounts.getcount(prev+[w])
        n_prev = self.rawcounts.getcount(prev)
        return (n_seq + self.labda)/(n_prev+self.lV)

In [32]:
lidstone_model1 = lidstone(vocab,ngram_cnt,1,0.5)

print(lidstone_model1.prob('testament',['the','old']))
print(lidstone_model1.prob('man',['the','old']))
print(lidstone_model1.prob('tower',['the','old']))
print(lidstone_model1.prob('twer',['the','ldpuio']))

2.508297575921704e-05
0.0010258511950337408
1.062837955899027e-05
4.251351823596108e-07


In [33]:
lidstone_model2 = lidstone(vocab,ngram_cnt,2,0.5)

print(lidstone_model2.prob('testament',['the','old']))
print(lidstone_model2.prob('man',['the','old']))
print(lidstone_model2.prob('tower',['the','old']))
print(lidstone_model2.prob('twer',['the','ldpuio']))

0.0002933411557641537
0.0026400704018773834
1.955607705094358e-05
2.007427481682224e-05


In [34]:
lidstone_model3 = lidstone(vocab,ngram_cnt,3,0.5)

print(lidstone_model3.prob('testament',['the','old']))
print(lidstone_model3.prob('man',['the','old']))
print(lidstone_model3.prob('tower',['the','old']))
print(lidstone_model3.prob('twer',['the','ldpuio']))

0.0002187705097352877
0.001451840655516
1.9888228157753426e-05
2.007427481682224e-05


### Good Turing Discounting

In [35]:
class goodturing(langmodel):
    
    def __init__(self,vocab,counts,n):
            self.n = n
            self.rawcounts = counts
            self.V = len(vocab) 
            N_tg = 30 * [0]
            nrbg = 0
            nrtg = 0
            for (ng,cnt) in counts.allngrams(self.n):
                if len(ng) == self.n:
                    nrtg += 1
                    if cnt < 30: 
                        N_tg[cnt]+=1
            
            N_tg[0] = self.V**self.n - nrtg

            self.reestimated = []
            for i in range(10):
                self.reestimated.append((i + 1)*N_tg[i + 1]/N_tg[i])
            
            print("Reestimates:")
            for i in range(10):
                print(i, '\t', N_tg[i], '\t',self.reestimated[i])
                
            ##New (n-1)gram counts
            ##based on summing up all new values for each n-gram
            self.default = self.V * self.reestimated[0]
            self.adjcounts = ngramcounts()
            for (ng,cnt) in counts.allngrams(self.n-1):
                if len(ng) == self.n - 1:
                    successors = counts.getsuccessors(ng)
                    missing = self.V - len(successors)
                    adjustedcnt =  missing * self.reestimated[0]
                    for cnt in successors.values():
                        if cnt < 10:
                            cnt = self.reestimated[cnt]
                        adjustedcnt += cnt
                    self.adjcounts.add(ng,adjustedcnt)
        
    def prob(self,w,prev):
        prev = prev[-self.n+1:]
        n_seq = self.rawcounts.getcount(prev+[w])
        if n_seq < 10:
            n_seq = self.reestimated[n_seq]
        n_prev = self.adjcounts.getcount(prev)
        if n_prev == 0:
            n_prev = self.default
            
                        
        return n_seq/n_prev    

In [36]:
gtmodel2 = goodturing(vocab,ngram_cnt,2)

Reestimates:
0 	 2481101319 	 0.00012938367229976068
1 	 321014 	 0.3166341654881096
2 	 50822 	 1.177875723112038
3 	 19954 	 2.0765761250877017
4 	 10359 	 3.1224056376098077
5 	 6469 	 4.055031689596538
6 	 4372 	 4.958600182982616
7 	 3097 	 6.111721020342267
8 	 2366 	 6.808960270498732
9 	 1790 	 7.905027932960894


Just to give some inside, we already printed the reestimates for n-grams. What frequency is assumed for a (n-1)-gram that was never seen before? If we build a bigram model, i.e. what is the assumed freqeuncy of an unseen word?

In [37]:
gtmodel2.default

6.445247635612578

For the word we have seen the counts are changed in the following way: 

In [38]:
for ng,cnt in gtmodel2.adjcounts.allngrams(1):
    print(ng,cnt,ngram_cnt.getcount(ng),sep = '\t')

['have']	3262.7665580435514	3918
['a']	18243.137914954226	22988
['chance']	110.98246940692452	131
['to']	23021.239570644157	25893
['compete']	20.997619077906723	23
['with']	5877.092846767324	7254
['large']	227.41718618245298	359
['supermarkets']	7.394761981060009	3
['is']	8684.890817068841	10054
['on']	5826.184281165417	6673
['sunday']	69.37473717360669	94
["''"]	8285.217609135452	8690
[',']	51389.79607065993	57930
['the']	59611.063269553815	69296
["council's"]	9.433749878123674	6
['resolution']	40.68901060382083	52
['said']	1698.9190848491262	1885
['.']	45314.92068061865	48969
['small']	314.0060670634207	542
['shops']	14.84294409913465	17
['``']	6861.375128337675	8734
['must']	829.267145396524	1006
['be']	4963.80546352655	6308
['retained']	16.16949555895272	22
['for']	7716.246810528566	9379
['they']	3054.083327045376	3604
['provide']	143.75943799438892	209
['essential']	52.436661613564176	80
['service']	243.1363330615112	313
['community']	169.004845933695	228
['according']	142.7111373

['action']	238.05308904462774	287
['new']	1116.7096121280968	1618
['ordinance']	10.928000999379222	9
['permitting']	8.977285890139056	8
['motorists']	8.027771544691628	5
['plead']	8.572508320499745	5
['guilty']	26.8088890654513	29
['minor']	31.29572356081376	58
['traffic']	44.145040738604855	68
['offenses']	8.027771544691628	5
['pay']	112.59676202601784	157
['fines']	7.078257199244199	2
['police']	104.09502350472188	151
['station']	75.59434100265531	104
['may']	1136.9161552598605	1395
['taken']	213.80168493101206	276
["monday's"]	7.394761981060009	3
['north']	145.49936479464444	205
['frank']	37.501988987575984	67
['sanantonio']	6.761752417428388	1
['ask']	92.85365013652985	125
['formally']	11.825828926481341	17
['request']	41.350247601558706	48
['michael']	10.471537011394602	11
['abatuno']	6.761752417428388	1
['draft']	18.8433502041568	24
['session']	65.22146841023405	74
['authorized']	33.05840478505865	37
['adopt']	11.649283350834343	13
['such']	904.3033350207801	1290
['means']	250.78

['political']	136.62627508649842	253
['try']	126.35123810786367	140
['make']	639.9335941817944	786
['name']	244.29686877389778	293
['condemning']	8.256003538683936	4
['stature']	14.720745661300082	15
['rebound']	7.078257199244199	2
['discredit']	7.078257199244199	2
['sees']	23.84459256768772	36
['ahead']	95.9153871031428	109
['charles']	48.75794687652343	92
['w.']	43.41124699755322	81
['sandman']	8.572508320499745	5
['r-cape']	6.761752417428388	1
['today']	228.65483139664843	281
['run']	162.05935776519036	210
['well']	757.9239022081125	895
['opponents']	11.332778569018533	12
['nomination']	12.459710964986503	12
['addressing']	10.420723067922838	9
['military']	119.44151812491901	207
['newark']	10.648955061915146	8
['essex']	12.497169809338237	12
['county']	81.61467904379106	133
['leaders']	83.3423836922262	107
['managers']	19.785694645226588	24
['gathering']	18.10598309419932	28
['reports']	68.29013322958134	82
['level']	181.9691519010633	212
['indicate']	66.03989791300417	80
['chosen']

['honored']	19.785694645226588	24
['colleagues']	18.455968897567754	23
['outgoing']	8.660781108323247	7
['whose']	105.73120791118274	251
['four-year']	7.394761981060009	3
['terms']	143.867232569857	163
['expire']	6.761752417428388	1
['april']	45.37030116738638	69
['18']	43.00890162285735	53
['carved']	11.965788132650149	14
['wooden']	26.703298621571527	50
['elephants']	12.082673850615864	10
['complete']	102.21600244063609	181
['ivory']	12.91530247809758	17
['tusks']	7.394761981060009	3
['remember']	104.0057175312387	138
['1961']	104.09964609360964	132
['retirements']	6.761752417428388	1
['leaving']	56.89130855037495	88
['choice']	94.4947759089496	111
['voters']	13.409225310433932	15
['presentation']	31.775518583552035	33
['geraldine']	7.078257199244199	2
['thompson']	25.129630231964995	38
['bank']	63.424879344206936	81
['stepping']	11.150047798629133	9
['down']	801.9588945513165	892
['35']	12.95276132244931	17
['she']	2450.0113564146213	2860
['1920s']	8.256003538683936	4
['following']	

['official']	35.839165730009434	73
['$40,000,000']	6.761752417428388	1
['budget']	40.245294825938544	57
['submitted']	19.04030985782064	21
['congressional']	16.08122277112922	20
['approval']	43.325813439872384	48
['$26,000,000']	6.761752417428388	1
['universities']	25.63314745742951	32
['twelve']	32.14684633900866	48
['contract-negotiation']	6.761752417428388	1
['stage']	135.84515563836376	175
['gordon']	9.610295453770675	10
['boyce']	6.761752417428388	1
['director']	79.96696588447867	90
['interview']	25.396397405480016	33
['six']	141.50211682064162	218
['agriculture']	20.2501416778102	22
['southeast']	24.371637082409908	26
['asia']	37.30659060989404	43
['africa']	41.062758606991956	44
['middle']	100.32675382857751	118
['east']	140.1718215336208	178
['latin']	36.904966208270416	49
['america']	154.84485562736495	193
['raised']	65.30522715359167	100
['church']	269.60939899487977	347
['disclosed']	13.032188196063291	13
['confusion']	35.657380380887695	44
['methods']	119.88589163090363	140

...

['cohnfidunt']	6.761752417428388	1
['coahse']	6.761752417428388	1
["cain't"]	6.761752417428388	1
['whah']	6.761752417428388	1
['nawt']	6.761752417428388	1
['coudn']	6.761752417428388	1
['ansuh']	6.761752417428388	1
['aw']	7.939498756868127	3
['gay-ess']	6.761752417428388	1
['fathuh']	6.761752417428388	1
['uttuh']	6.761752417428388	1
['wohd']	6.761752417428388	1
['aftuh']	6.761752417428388	1
['majuh']	6.761752417428388	1
["t'gethuh"]	6.761752417428388	1
["f'ovuh"]	6.761752417428388	1
['behahn']	6.761752417428388	1
['doan']	6.761752417428388	1
["thet's"]	6.761752417428388	1
['nahce']	6.761752417428388	1
["d'you"]	6.761752417428388	1
['taos']	6.761752417428388	1
['eventshahleh']	6.761752417428388	1
['eventshah-leh']	6.761752417428388	1
['crannies']	7.078257199244199	2
['re-echo']	6.761752417428388	1
['senor']	7.078257199244199	2
["moriarty's"]	6.761752417428388	1
['dust-swirling']	6.761752417428388	1
['shards']	6.761752417428388	1
['unwired']	6.761752417428388	1
['loosely-taped']	6.761752

['emanated']	6.761752417428388	1
['prieur']	8.344276326507439	6
['propelled']	6.761752417428388	1
['coiled']	6.761752417428388	1
['luxuriosly-upholstered']	6.761752417428388	1
['inductees']	6.761752417428388	1
['gambits']	6.761752417428388	1
['liveried']	6.761752417428388	1
['proprietory']	6.761752417428388	1
['aristide']	7.078257199244199	2
['chamois']	6.761752417428388	1
["louis'"]	6.761752417428388	1
['bleedings']	6.761752417428388	1
['devol']	7.394761981060009	3
['leone']	6.761752417428388	1
['well-brushed']	6.761752417428388	1
['coachman']	7.394761981060009	3
['gloved']	7.078257199244199	2
['cowering']	6.761752417428388	1
['677']	6.761752417428388	1
['perdido']	8.52169437702798	3
['pint-sized']	6.761752417428388	1
['myopic']	6.761752417428388	1
['blurry']	6.761752417428388	1
['flee']	6.761752417428388	1
["lalauries'"]	6.761752417428388	1
["delphine's"]	7.078257199244199	2
['endearments']	7.078257199244199	2
['drudgery']	6.761752417428388	1
['trustingly']	6.761752417428388	1
['asso

['too-shiny']	6.761752417428388	1
['dawns']	6.761752417428388	1
["shafer's"]	6.761752417428388	1
['whitewashed']	6.761752417428388	1
['spellbound']	6.761752417428388	1
['rehearsing']	6.761752417428388	1
['bumps']	6.761752417428388	1
['rippled']	6.761752417428388	1
["rat's"]	6.761752417428388	1
['factory-to-you']	6.761752417428388	1
['unventilated']	6.761752417428388	1
['malingering']	6.761752417428388	1
['primping']	6.761752417428388	1
['boxed-in']	6.761752417428388	1
['whichever-the-hell']	6.761752417428388	1
['six-thirty']	6.761752417428388	1
['eight-by-ten']	6.761752417428388	1
["jeweler's"]	6.761752417428388	1
['magpie']	6.761752417428388	1
['fifteenth-century']	6.761752417428388	1
['ferreted']	6.761752417428388	1
['mckenzie']	7.939498756868127	3
['rufus']	6.761752417428388	1
['researches']	6.761752417428388	1
['haircuts']	6.761752417428388	1
['fauntleroy']	6.761752417428388	1
['monmouth']	7.394761981060009	3
['nearsighted']	6.761752417428388	1
['sleepwalker']	6.761752417428388	1
[

['incarcerated']	6.761752417428388	1
['dusted']	6.761752417428388	1
['tea-drinking']	6.761752417428388	1
['ryusenji']	11.193691837723266	8
['fudomae']	7.939498756868127	3
["charlotte's"]	7.394761981060009	3
['flippant']	6.761752417428388	1
['pointless']	6.761752417428388	1
['soba']	6.761752417428388	1
['udon']	6.761752417428388	1
['sashimi']	6.761752417428388	1
['ceecee']	7.078257199244199	2
['witter']	6.761752417428388	1
['gompachi']	6.761752417428388	1
['komurasaki']	6.761752417428388	1
['parkish']	6.761752417428388	1
['kanto']	6.761752417428388	1
['fudo']	9.787713504291219	7
['bothersome']	6.761752417428388	1
['ascetic']	6.761752417428388	1
['spigots']	6.761752417428388	1
['caged']	6.761752417428388	1
['limpid']	6.761752417428388	1
['fountain-falls']	6.761752417428388	1
['hundred-yen']	6.761752417428388	1
["fudo's"]	7.078257199244199	2
['stubble']	7.078257199244199	2
['snow-fence']	6.761752417428388	1
['lath']	7.078257199244199	2
['furrow']	9.699440716467718	5
['multi-colored']	6.76

['aquam']	6.761752417428388	1
['silvas']	6.761752417428388	1
['agricolas']	6.761752417428388	1
['portant']	6.761752417428388	1
['vignette']	6.761752417428388	1
['pre-punic']	6.761752417428388	1
['fifty-third']	6.761752417428388	1
['affianced']	6.761752417428388	1
['cancels']	6.761752417428388	1
['buckling']	6.761752417428388	1
['scimitar']	6.761752417428388	1
['stumbles']	6.761752417428388	1
["opera's"]	6.761752417428388	1
["spumoni's"]	6.761752417428388	1
['sevigli']	6.761752417428388	1
['spegititgninino']	6.761752417428388	1
['contralto']	6.761752417428388	1
['hattie']	6.761752417428388	1
['sforzt']	6.761752417428388	1
['prometheus']	6.761752417428388	1
['conduit']	6.761752417428388	1
['hubba']	7.078257199244199	2
['yalagaloo']	6.761752417428388	1
['ranavan']	6.761752417428388	1
['armadillo']	7.622993975052316	2
['atonally']	6.761752417428388	1
['grunnfeu']	6.761752417428388	1
['arapacis']	7.078257199244199	2
['serbantian']	6.761752417428388	1
['invocation']	6.761752417428388	1
['phi

In [39]:
print(gtmodel2.prob('testament',['the','old']))
print(gtmodel2.prob('man',['the','old']))
print(gtmodel2.prob('tower',['the','old']))
print(gtmodel2.prob('twer',['the','ldpuio']))

0.015264043217479543
0.167332718912922
3.231361443113055e-07
2.0074274816822244e-05


In [40]:
gtmodel3 = goodturing(vocab,ngram_cnt,3)

Reestimates:
0 	 123617626530634 	 6.436193788293077e-09
1 	 795627 	 0.13678017462956887
2 	 54413 	 0.888206862330693
3 	 16110 	 1.7450031036623215
4 	 7028 	 2.736909504837792
5 	 3847 	 3.855471796204835
6 	 2472 	 4.652508090614886
7 	 1643 	 5.541083384053561
8 	 1138 	 6.825131810193322
9 	 863 	 7.404403244495945


In [41]:
print(gtmodel3.prob('testament',['the','old']))
print(gtmodel3.prob('man',['the','old']))
print(gtmodel3.prob('tower',['the','old']))
print(gtmodel3.prob('twer',['the','ldpuio']))

0.037888259450229576
0.3537770245267795
6.324937465833516e-11
2.0074274816822244e-05


Note that the oov word following an oov word is more likely than a frequent word in an unseen n-gram!

### Katz' Backoff

In [42]:
# Backs off only once, so wmorks only with bigrams
class backoff(langmodel):
    
    def __init__(self,vocab,counts,n):
            self.n = n
            self.rawcounts = counts
            self.V = len(vocab) 
            N_tg = 30 * [0]
            nrbg = 0
            nrtg = 0
            for (ng,cnt) in counts.allngrams(self.n):
                if len(ng) == self.n:
                    nrtg += 1
                    if cnt < 30: 
                        N_tg[cnt]+=1
            
            N_tg[0] = self.V**self.n - nrtg

            self.reestimated = []
            for i in range(10):
                self.reestimated.append((i + 1)*N_tg[i + 1]/N_tg[i])
                
            ##New (n-1)gram counts  and alpha's for adjusting backoff probabilities
            self.default = self.V * self.reestimated[0]
            self.adjcounts = ngramcounts()
            self.alpha = {}
            for (ng,cnt) in counts.allngrams(self.n - 1):
                if len(ng) == self.n - 1:
                    successors = counts.getsuccessors(ng)
                    missing = self.V - len(successors)
                    missingcnt =  missing * self.reestimated[0]
                    adjustedcnt = missingcnt
                    for cnt in successors.values():
                        if cnt < 10:
                            cnt = self.reestimated[cnt]
                        adjustedcnt += cnt
                    self.adjcounts.add(ng,adjustedcnt)
                    self.alpha[tuple(ng)] = missingcnt/adjustedcnt
        
    def prob(self,w,prev):
        prev = prev[-self.n+1:]
        n_seq = self.rawcounts.getcount(prev+[w])
        if n_seq > 0:
            if n_seq < 10:
                n_seq = self.reestimated[n_seq]
            n_prev = self.adjcounts.getcount(prev)
            if n_prev == 0:
                n_prev = self.default            
        
            return n_seq/n_prev
        else: #backoff to n-1 model
            if len(prev) > 1:
                prev = prev[1:]
            else:
                prev = []
            n_seq = 0.5 + self.rawcounts.getcount(prev+[w])
            n_prev = 0.5 * self.V + self.rawcounts.getcount(prev)
            alpha = self.alpha.get(tuple(prev+[w]),1)
            return alpha * n_seq/n_prev

In [43]:
bo_model2 = backoff(vocab,ngram_cnt,2)

How much probability mass is reserved for unknown bigrams?

In [44]:
print(bo_model2.alpha[('old',)])
print(bo_model2.alpha[('varicolored',)])

0.01597488156631801
0.9531727655878103


In [45]:
print(bo_model2.prob('testament',['the','old']))
print(bo_model2.prob('man',['the','old']))
print(bo_model2.prob('tower',['the','old']))
print(bo_model2.prob('twer',['the','ldpuio']))

0.015264043217479543
0.167332718912922
6.348456282355263e-06
4.251351823596108e-07


## Evaluation

We compute the perpexity of the test data to evaluate the models.

In [46]:
import math

def evaluate(model,text):
    entropy = 0
    for i in range(2,len(text)):
        entropy -= math.log(model.prob(text[i],text[i-2:i]))
    entropy /= (i-2)
    return math.exp(entropy)

In [47]:
evaluate(eq_model,testtext)

49868.92559539385

Do you remember the vocabulary size?

In [48]:
models = {"Equal probabilities":eq_model,
          "Laplace - bigrams":laplace_model2,
          "Laplace - trigrams":laplace_model3,
          "Lidstone - monograms":lidstone_model1,
          "Lidstone - bigrams":lidstone_model2,
          "Good Turing - bigrams":gtmodel2,
          "Back-Off - bigrams":bo_model2}

In [49]:
print("{:<25} {:<15}".format('Model','Perplexity'))
print("{:<25} {:<15}".format('------','------------'))
for name in models: 
    p = evaluate(models[name],testtext)
    print ("{:<25} {:>9.2f}".format(name,p))


Model                     Perplexity     
------                    ------------   
Equal probabilities        49868.93
Laplace - bigrams           6567.25
Laplace - trigrams         31718.68
Lidstone - monograms        1454.27
Lidstone - bigrams          4915.19
Good Turing - bigrams       2067.65
Back-Off - bigrams          1132.69


## A simple application: Auto Completion

Typical applicayions of language models combine two typs of information: the probability that a word is generated from a source, like speech signal, misspelled word, translation, etc. and the probability that the word follows the previous word, as computed by the language model.  A very easy application is autocompletion: we predict the next word, but it has to fit to what was already typed. 

For an efficient implementation we should should store our vocabulary in a prefix tree, in order to get fast access to all words starting with a given prefix. Here we will simply iterate throught the whole dictionary, ignoring efficiency issues.

In [50]:
def predict(model,words,beginning):
    maxprob = 0
    bestword = ""
    
    for w in vocab:
        if w.startswith(beginning):
            prob = model.prob(w,words)
            if prob > maxprob:
                maxprob = prob
                bestword = w
    return bestword

In [51]:
predict(bo_model2,['the','old'],'tow')

'town'

In [52]:
predict(lidstone_model1,['the','old'],'tow')

'toward'

In [53]:
predict(lidstone_model2,['the','old'],'tow')

'town'

In [54]:
prev = ['old']
test = 'testament'
for i in range(len(test)+1):
    print(str(i)+' '+prev[0]+' '+test[:i]+'\t'+predict(lidstone_model1,prev,test[:i]))
 
print()

prev = ['old']
test = 'testament'
for i in range(len(test)+1):
    print(str(i)+' '+prev[0]+' '+test[:i]+'\t'+predict(bo_model2,prev,test[:i]))


0 old 	the
1 old t	the
2 old te	tell
3 old tes	test
4 old test	test
5 old testa	testament
6 old testam	testament
7 old testame	testament
8 old testamen	testament
9 old testament	testament

0 old 	man
1 old t	testament
2 old te	testament
3 old tes	testament
4 old test	testament
5 old testa	testament
6 old testam	testament
7 old testame	testament
8 old testamen	testament
9 old testament	testament


### Evaluation

How much letters do we save typing on average? We just count. However, saving 1 letter is not helpfull and does not really save time. Maybe we should only count the longer savings.

In [55]:
def test_pred(model,word,prevwords):
    saving = 0
    for i in range(len(word)-1,-1,-1):
        if predict(model,prevwords,word[:i]) == word:
            saving = len(word) - i
        else:
            break
    return saving    

In [56]:
test_pred(lidstone_model1,'testament',['old'])

4

In [57]:
test_pred(gtmodel2,'testament',['old'])

8

In [58]:
def eval_ac(model,text):
    count = 0
    saving = 0
    for i in range(2,len(text)):
            prec = [w.lower() for w in text[i-2:i]]
            word = text[i].lower()
            count += 1
            saving += test_pred(model,word,prec)
    return float(saving)/float(count)

In [59]:
print("{:<25} {:<15}".format('Model','Average saving'))
print("{:<25} {:<15}".format('------','------------'))
for name in models: 
    p = eval_ac(models[name],testtext[:1000])
    print ("{:<25} {:>9.2f}".format(name,p))

Model                     Average saving 
------                    ------------   
Equal probabilities            0.54
Laplace - bigrams              1.66
Laplace - trigrams             0.99
Lidstone - monograms           1.51
Lidstone - bigrams             1.66
Good Turing - bigrams          1.66
Back-Off - bigrams             1.81


1. Retrieve the POS tags for all words in the Brown corpus using nltk.corpus.brown.tagged_words() (including the test data!). For each word collect the 5 most frequent POS tags. Assign a list of these POS tags ordered by the frequency to each word.

In [60]:
print(len(nltk.corpus.brown.tagged_words()))


1161192


In [61]:
data=nltk.corpus.brown.tagged_words()
tagged_word=nltk.corpus.brown.tagged_words()

In [64]:
from collections import defaultdict
word_tags = defaultdict(Counter)
for word, pos in brown.tagged_words():
    word_tags[word][pos] +=1

In [68]:
print(word_tags["go"])
print(word_tags["come"])
print(word_tags["apple"])
print(word_tags["time"])

Counter({'VB': 595, 'VB-NC': 8, 'NN': 1, 'VB-HL': 1})
Counter({'VB': 395, 'VBN': 191, 'VB-NC': 2, 'VB-HL': 1})
Counter({'NN': 7, 'NN-HL': 1})
Counter({'NN': 1547, 'NN-HL': 8, 'VB': 1})


In [66]:
word_freq={}
for key in word_tags.keys():
    if len(word_tags[key])>5:
        word_freq[key]=list(word_tags[key])[:5]
    else:
        word_freq[key]=list(word_tags[key]) 

2. Make classes of all words having the same sequence of possible POS tags, eventually, merge classes that consists of only a few infrequent words, e.g. by merging classes with the same set of labels, but withe a different order.


In [73]:

classes = {}
  
for key, value in word_freq.items():
    value=tuple(value)
    if value not in classes:
        classes[value] = [key]
    else:
        classes[value].append(key)
  


In [71]:
print(len(word_freq))
print(len(classes))

56057
1581


In [72]:
print("Dictionary", str(classes))

Dictionary {('AT', 'AT-TL', 'AT-HL'): ['The'], ('NP-TL', 'NP'): ['Fulton', 'Atlanta', 'Grady', 'Miller', 'Beaumont', 'Lamar', 'Wise', 'Oklahoma', 'Massachusetts', 'York', 'Lao', 'America', 'Asia', 'Narragansett', 'Westfield', 'Essex', 'Michigan', 'Orleans', 'Miles', 'Pennsylvania', 'Columbia', 'Lafayette', 'Montgomery', 'Eastwick', 'Multnomah', 'Kiwanis', 'Beaverton', 'Florida', 'Columbus', 'Franklin', 'Louisiana', 'Minnesota', 'Ben', 'Babe', 'Ruth', 'Slocum', 'Mercer', 'Georgetown', 'Hogan', 'Warwick', 'Billiken', 'Arab', 'Brandenburg', 'Colorado', 'Perkins', 'Arlington', 'Philmont', 'Delaware', 'Drexel', 'Wellesley', 'Bailey', 'Westinghouse', 'Wabash', 'Dunbar', 'Jenks', 'Sterling', 'Elliott', 'McClellan', 'Delhi', 'Emory', 'Waddell', 'Leon', 'Cain', 'Piedmont', 'Burnside', 'Schuyler', 'Bess', 'Bond', 'Nevada', 'Beverly', 'Fletcher', 'Aurora', 'Guerin', 'Hardwicke-Etter', 'Moss', 'Gordin', 'Stacy', 'Mexico', 'Forsythe', 'Morocco', 'Cadillac', 'Hilton', 'Wilshire', 'Judson', 'Mackey',

3. Write a function that computes the probabilities p( wi | ci) on test and training data.

In [75]:
word_freq={}
dict_new={}
for key in word_tags.keys():
    if len(word_tags[key])>5:
        word_freq[key]={A:N for (A,N) in [x for x in word_tags[key].items()][:4]}
    else:
        word_freq[key]=dict(word_tags[key])

for key,val in brown.tagged_words():
    if val in dict_new.keys():
        dict_new[val]+=1
    else:
        dict_new[val]=1       